# Output In The Construction Industry

-----

### Requirements

We're looking to transform **tabs 1 and 2 only**.

#### Observations & Dimensions

The `observations` should be apparent.

The required dimensions are:

* **Geography** - it's all UK level data (the code for UK is "K02000001")
* **Time** - either a simple year, or a year followed by a quarter, i.e "2002 Q2"
* **Adjustment** - either seasonal or non-seasonal
* **Business** - concatenation of rows 6-9 around hyphens. i.e 'Other New Work - Excluding Infrastructure - Private Commercial'
* **CDID** - ONS specific 4 letter codes. Typically on row 10 

-----
Notes:

* Getting the **Business** dimension cleanly is going to be tricky (read - cant see an obvious way to do it), I'd perhaps leave this one until last.
* It's always worth getting the file out of /sources and having a look over.
* You can't really take CDID as a dimension (dimension items needs to be repeating, not unqiue), it's a good exercise though as if doing this for real we'd likely be taking it as meta/supporting data.

In [37]:
from databaker.framework import *
import pandas as pd


list_tabs = ["Table 1a", "Table 1b", "Table 2a", "Table 2b"]
tabs = loadxlstabs("./sources/OIC.xls", list_tabs) # load tabs

Loading ./sources/OIC.xls which has size 759808 bytes
Table names: ['Table 1a', 'Table 1b', 'Table 2a', 'Table 2b']


In [38]:
tidied_sheets = []

for tab in tabs:
    
    observations = tab.excel_ref("G11").expand(DOWN).expand(RIGHT)\
    .is_not_blank() - tab.excel_ref("J11").expand(DOWN).expand(RIGHT).is_not_blank()
    
    geography = "K02000001"  #constabt
    
    #quarter = tab.excel_ref("B11").expand(DOWN).is_not_blank()
    time = tab.excel_ref("A11").expand(DOWN).is_not_blank()
    adjustment = tab.excel_ref("C2")
    
    business1 = tab.excel_ref("F6")
    business2 = tab.excel_ref("G7")
    business = tab.excel_ref("G8").expand(RIGHT).is_not_blank() -  tab.excel_ref("J8").expand(RIGHT).is_not_blank()
    
    cdid = tab.excel_ref("G10").expand(RIGHT).is_not_blank()
    
    #observations = month.waffle(purpose)
    
    dimensions = [
        HDimConst("Geography", geography),
        HDimConst("Adjustment", adjustment.value),
        HDim(time, "Year", CLOSEST, UP),
        HDim(business1, "Biz1", CLOSEST, UP), # Other new work
        HDim(business2, "Biz2", CLOSEST, UP), #Excluding infrastructure
        HDim(business, "Biz", DIRECTLY, ABOVE), 
        HDim(cdid, 'ONS Code', DIRECTLY, ABOVE)
            ]
    cs = ConversionSegment(tab, dimensions, observations) # < --- processing
    tidy_sheet = cs.topandas() #dataframe
    tidied_sheets.append(tidy_sheet) # <-- adding result of processing this tab to our list

In [39]:
datacube = pd.concat(tidied_sheets)

In [40]:
datacube

,OBS,Geography,Adjustment,Year,Biz1,Biz2,Biz,ONS Code
0,50.0,K02000001,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME SEASONALLY ADJUSTED INDEX NUMBERS - BY SECTOR,1997.0,Other New Work,Excluding Infrastructure,Public,MV39
1,175.6,K02000001,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME SEASONALLY ADJUSTED INDEX NUMBERS - BY SECTOR,1997.0,Other New Work,Excluding Infrastructure,Private Industrial,MV3A
2,81.6,K02000001,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME SEASONALLY ADJUSTED INDEX NUMBERS - BY SECTOR,1997.0,Other New Work,Excluding Infrastructure,Private Commercial,MV3B
3,52.7,K02000001,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME SEASONALLY ADJUSTED INDEX NUMBERS - BY SECTOR,1998.0,Other New Work,Excluding Infrastructure,Public,MV39
4,178.9,K02000001,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME SEASONALLY ADJUSTED INDEX NUMBERS - BY SECTOR,1998.0,Other New Work,Excluding Infrastructure,Private Industrial,MV3A
...,...,...,...,...,...,...,...,...
676,403.0,K02000001,"CONSTRUCTION OUTPUT: VOLUME NON-SEASONALLY ADJUSTED, BY SECTOR",2019.0,Other New Work,Excluding Infrastructure,Private Industrial,MV4F
677,2313.0,K02000001,"CONSTRUCTION OUTPUT: VOLUME NON-SEASONALLY ADJUSTED, BY SECTOR",2019.0,Other New Work,Excluding Infrastructure,Private Commercial,MV4G
678,846.0,K02000001,"CONSTRUCTION OUTPUT: VOLUME NON-SEASONALLY ADJUSTED, BY SECTOR",2019.0,Other New Work,Excluding Infrastructure,Public,MV4E
679,413.0,K02000001,"CONSTRUCTION OUTPUT: VOLUME NON-SEASONALLY ADJUSTED, BY SECTOR",2019.0,Other New Work,Excluding Infrastructure,Private Industrial,MV4F


In [42]:
adjustment_dict = {"NON-SEASONALLY": "Non-seasonal",
           "SEASONALLY": "Seasonal"}
for key,value in adjustment_dict.items():
    datacube.loc[datacube["Adjustment"].str.contains(key),"Adjustment"] = value

In [43]:
datacube["OBS"] = datacube["OBS"].astype(float).astype(int)

In [44]:
datacube["Year"] = datacube["Year"].astype(float).astype(int)

In [45]:
datacube["Business"] = datacube["Biz1"] + "-" + datacube["Biz2"] + "-" + datacube["Biz"]

In [46]:
datacube.drop(columns = ['Biz1', 'Biz2', "Biz"], inplace=True)

In [47]:
datacube

,OBS,Geography,Adjustment,Year,ONS Code,Business
0,50,K02000001,Seasonal,1997,MV39,Other New Work-Excluding Infrastructure-Public
1,175,K02000001,Seasonal,1997,MV3A,Other New Work-Excluding Infrastructure-Private Industrial
2,81,K02000001,Seasonal,1997,MV3B,Other New Work-Excluding Infrastructure-Private Commercial
3,52,K02000001,Seasonal,1998,MV39,Other New Work-Excluding Infrastructure-Public
4,178,K02000001,Seasonal,1998,MV3A,Other New Work-Excluding Infrastructure-Private Industrial
...,...,...,...,...,...,...
676,403,K02000001,Non-seasonal,2019,MV4F,Other New Work-Excluding Infrastructure-Private Industrial
677,2313,K02000001,Non-seasonal,2019,MV4G,Other New Work-Excluding Infrastructure-Private Commercial
678,846,K02000001,Non-seasonal,2019,MV4E,Other New Work-Excluding Infrastructure-Public
679,413,K02000001,Non-seasonal,2019,MV4F,Other New Work-Excluding Infrastructure-Private Industrial
